In [ ]:
import numpy as np
import random
from scipy import ndimage, misc
import copy
import time
from deap import base
from deap import creator
from deap import tools
import game


In [ ]:
n_weights = 9
population_size = 30
crossover_probability = 1
mutation_probability = 0.1

moves = [0,1,2,3]
directions = {
        0: "down",
        1: "up",
        2: "left",
        3: "right"
    }

generations = 2000
gameIterations = 2000

In [ ]:
''' DEAP SETUP PT 1'''

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("GameInstance", game.Game)
creator.create("Individual", list, 
                fitness=creator.FitnessMax, 
                gameInstance=creator.GameInstance)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.uniform, 1, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, n_weights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
''' Example '''

# Create individual
ind = toolbox.individual()
print(ind) # 10 weights between 0-1

# Game within Individual
print("---")
print(ind.gameInstance.grid)

# Make a move in the game
ind.gameInstance.move(0) # 0 => down 
print("---")
print(ind.gameInstance.grid)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
---
[[0 0 2 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
---
[[0 0 0 0]
 [0 0 2 0]
 [0 0 0 0]
 [2 0 2 0]]


In [ ]:

# Feature f2 is 1, if second or third highest value tile is next to highest
# value tile. Otherwise, it is 0.
def f10(newstate, currentstate):

    f1, f2, f3, f4, f5, f6, f7, f8, f9 = 0, 0, 0, 0, 0, 0, 0, 0, 0
    ###
    flatstate = newstate.flatten()
    ###
    maxval= max(flatstate)
    ###
    secondmaxval = max(np.where(flatstate == maxval, 1, flatstate)) 
    secondmaxval = secondmaxval if secondmaxval != 1 else 0
    ###
    thirdmaxval = max(np.where(flatstate == secondmaxval, 1, flatstate)) 
    thirdmaxval = thirdmaxval if thirdmaxval != 1 else 0
    ###
    maxvalues = [maxval, secondmaxval, thirdmaxval]
    ###
    f5cells = [[0,0],[0,3],[1,1],[2,2],[3,0]]
    f6cells = [[2,3],[3,2],[0,1],[0,2],[1,1],[2,0],[1,3],[3,1]]
    ###
    newstateT = newstate.T

    highestValuesIndeces = [0,0]
    for nrow in range(4):

        for ncolumn in range(4):
            val = newstate[nrow,ncolumn]

            if val != 0: f4 += 1  
            if  val == maxval and (nrow,ncolumn) in [(0,0), (0,3), (3,3), (3,0)]: f1 = 1

            if val == secondmaxval: highestValuesIndeces[1] = [nrow,ncolumn]
            if val == maxval: highestValuesIndeces[0] = [nrow,ncolumn]

            if currentstate[nrow,ncolumn] != val: f3 += 1
            if val == maxval:
                # Check above
                if ncolumn-1 >= 0: 
                    nearbyvalue = newstate[nrow,ncolumn-1]
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1

                # Check right
                if nrow+1 <= 3:
                    nearbyvalue = newstate[nrow+1,ncolumn]
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1
                # Check below
                if ncolumn+1 <= 3:
                    nearbyvalue = newstate[nrow,ncolumn+1] 
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1
                # Check left
                if nrow-1 >= 0:
                    nearbyvalue = newstate[nrow-1,ncolumn]
                    if nearbyvalue in maxvalues: f2 = 1
                    if nearbyvalue == val and [nrow, ncolumn] in f5cells: f5 += 1
                    if nearbyvalue == val and [nrow, ncolumn] in f6cells: f6 += 1
    
    if highestValuesIndeces[0][0] == highestValuesIndeces[1][0] and highestValuesIndeces[0][1] != highestValuesIndeces[1][1]:
        row = newstate[highestValuesIndeces[0][0]]
        if 0 not in newstate: f7 = 1
        if np.array_equiv(np.sort(row), row) or np.array_equiv(np.sort(row)[::-1], row): f8 = 1
        f9 = sum([1 for elem in row if elem != 0])/f4

    elif highestValuesIndeces[0][1] == highestValuesIndeces[1][1] and highestValuesIndeces[0][0] != highestValuesIndeces[1][0]:
        column = newstate[highestValuesIndeces[0][1]]
        if 0 not in newstateT: f7 = 1
        if np.array_equiv(np.sort(column), column) or np.array_equiv(np.sort(column)[::-1], column): f8 = 1
        f9 = sum([1 for elem in column if elem != 0])/f4
        
    else:
        f8 = 0
        f7 = 0

    return f1,f2,f3,f4,f5,f6,f7,f8,f9


In [ ]:
st = ind.gameInstance.project(1)
f10(st, ind.gameInstance.grid)

(0, 0, 4, 2, 0, 0, 0, 0, 1.0)

In [ ]:
# The state evaluation function
def stateEvaluation(ind, state):
    f1,f2,f3,f4,f5,f6,f7,f8,f9 = f10(state, ind.gameInstance.grid)
    E = ind[0]*f1 + ind[1]*f2 + ind[2]*f3 + ind[3]*f4 + ind[4]*f5 + ind[5]*f6 + ind[6]*f7 + ind[7]*f8 + ind[8]*f9
    return E

In [ ]:
ind = toolbox.individual()

statesEvaluation = []

# Generate possible future states
for move in moves:
    state = ind.gameInstance.project(move)
    E = stateEvaluation(ind, state)
    print(E)
    statesEvaluation.append(E)

chosenMove = moves[np.argmax(statesEvaluation)]


5.0
9.0
5.0
6.0


In [ ]:
# Game Evaluation Function
def gameEvaluation(moves, individual, gameIterations):

    # Play game for 1000 moves
    for iter in range(gameIterations):
        statesEvaluation = []

        # Evaluate all possible moves (projections)
        for move in moves:
            state = ind.gameInstance.project(move)
            if np.array_equal(state, individual.gameInstance.grid): E = 0
            else: E = stateEvaluation(individual, state)
            statesEvaluation.append(E)
            
        # Choose the best possible move with maximum E
        chosenMove = moves[np.argmax(statesEvaluation)]

        try: individual.gameInstance.move(chosenMove)
        except: 
            fitness = (individual.gameInstance.score,)
            individual.gameInstance.start()
            # Return fitness
            return fitness

    # Return the fitness, 
    # Maximum value and number of zeros after 1000 game moves
    fitness = (individual.gameInstance.score,)
    individual.gameInstance.start()
    return (iter,)

In [ ]:
def smallIncrementMut(ind, indpb):
    for i in range(len(ind)):
        if random.random() > indpb:
            ind[i] += np.random.normal(0, 0.2, 1)[0]
    return ind,

In [ ]:
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=mutation_probability)
#toolbox.register("mutate", tools.mutPolynomialBounded, eta=0.5, low=0, up=10, indpb=0.1)
#toolbox.register("mutate", smallIncrementMut, indpb=0.1)
toolbox.register("mate", tools.cxTwoPoint)
#toolbox.register("mate", tools.cxSimulatedBinary, eta=1.1)
toolbox.register("select", tools.selTournament, k=population_size-1)
toolbox.register("gameEvaluation", gameEvaluation, gameIterations=gameIterations, moves=moves)

In [ ]:
def main():
    best = []
    average = []
    # Create the population
    population = toolbox.population(population_size)

    # Evaluate
    for individual in population:
        individual.fitness.values = toolbox.gameEvaluation(individual=individual)

    # Keep track of the no. of generations
    generations = 0
    
    # Begin evolution
    while generations < 1000:
                
        # Selection
        #selected = toolbox.select(population, len(population))
        offspring = [toolbox.clone(ind) for ind in population]
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random()> crossover_probability:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Mutation
        for mutant in offspring:
            toolbox.mutate(mutant)
            del mutant.fitness.values

        # Evaluate new individuals with no fitness
        for individual in offspring:
            if not individual.fitness.valid:
                individual.fitness.values = toolbox.gameEvaluation(individual=individual)
   
        population = tools.selBest(offspring+population,k=1) + toolbox.select(offspring, tournsize=3)
        
        best.append(tools.selBest(population, k=1)[0])
        avg = sum([ind.fitness.values[0] for ind in population])/len(population)
        average.append(average)
        print(generations, best[-1].fitness.values[0], avg, best[-1].gameInstance.highestnumber)
        # Increment the generation counter
        generations += 1    
    
    return tools.selBest(population,k=1), best, average
    

best performance so far tournsize=2, k=1, cxTwoPoint, mutGaussian

In [13]:
best_ind, best, average = main()

0 384.0 285.8666666666667 32
1 400.0 327.46666666666664 32
2 400.0 298.0 32
3 636.0 340.8 64
4 636.0 311.73333333333335 64
5 636.0 298.53333333333336 64
6 636.0 380.8 64
7 636.0 389.8666666666667 64
8 636.0 368.4 64
9 636.0 375.73333333333335 64
10 636.0 458.4 64
11 636.0 388.1333333333333 64
12 636.0 431.6 64
13 636.0 446.6666666666667 64
14 636.0 436.53333333333336 64
15 636.0 386.8 64
16 636.0 387.06666666666666 64
17 636.0 382.4 64
18 636.0 393.6 64
19 636.0 390.4 64
20 636.0 423.06666666666666 64
21 636.0 395.8666666666667 64
22 636.0 401.8666666666667 64
23 636.0 404.1333333333333 64
24 636.0 394.0 64
25 636.0 410.93333333333334 64
26 636.0 401.8666666666667 64
27 636.0 388.1333333333333 64
28 636.0 405.46666666666664 64
29 636.0 418.53333333333336 64
30 636.0 398.6666666666667 64
31 656.0 434.0 64
32 656.0 413.06666666666666 64
33 656.0 420.93333333333334 64
34 656.0 389.06666666666666 64
35 656.0 415.46666666666664 64
36 656.0 402.1333333333333 64
37 656.0 398.0 64
38 656.0 390

In [14]:
print(best_ind)

best_ind.gameInstance.start()
# Play game with best individual
for iter in range(100):
    statesEvaluation = []

    # Evaluate all possible moves (projections)
    for move in moves:
        state = best_ind.gameInstance.project(move)
        if np.array_equal(state, best_ind.gameInstance.grid): E = 0
        else: E = stateEvaluation(best_ind, state)
        statesEvaluation.append(E)
        
    # Choose the best possible move with maximum E
    chosenMove = moves[np.argmax(statesEvaluation)]
    try: best_ind.gameInstance.move(chosenMove)
    except: 
        print(best_ind.gameInstance.grid)
        break

print(best_ind.gameInstance.grid)


[[-5.807200065438404, 0.02445071173512564, -5.673655829823235, 0.3345052817402731, -6.939000793475827, 0.8539650937557061, 6.486106031085276, -6.91240620120434, -3.7576148309120088]]


AttributeError: 'list' object has no attribute 'gameInstance'